# Défi Grosses Data 2018

## Début Commun pour tous

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
from os import listdir
import imp
import Annex
imp.reload(Annex)

In [ ]:
X_train,X_test,Y_train,Y_test,scaler=Annex.get_data_raw(scale=True, 
                                                        add_dummies=True,
                                                        var_dummies=['insee','ddH10_rose4'],
                                                        TrainTestSplit=True,
                                                        sz_test=0.3,
                                                        impute_method='drop',
                                                        convert_month2int=True,
                                                        date_method='drop')

## Maintenant, faites votre vie!

## Submission

In [ ]:
df_TEST=Annex.load_test_set() # Charge le jeu de données test

ATTENTION - il faut effectuer le même traitement sur les données test que celui effectué sur les données d'apprentissage

In [ ]:
#df_TEST.shape, df_TEST.info()
#df_TEST=Annex.convert_month_to_int(df_TEST)
#df_dummies=pd.get_dummies(df_TEST[['insee']])
#df_TEST_full_qtt=pd.concat([df_TEST,df_dummies],axis=1)
#df_TEST_full_qtt.flir1SOL0=df_TEST_full_qtt.flir1SOL0.fillna(0)
#df_TEST_full_qtt.fllat1SOL0=df_TEST_full_qtt.fllat1SOL0.fillna(0)
#df_TEST_full_qtt.flsen1SOL0=df_TEST_full_qtt.flsen1SOL0.fillna(0)
#df_TEST_full_qtt.flvis1SOL0=df_TEST_full_qtt.flvis1SOL0.fillna(0)
#df_TEST_full_qtt.rr1SOL0=df_TEST_full_qtt.rr1SOL0.fillna(0)
#df_TEST_full_qtt=df_TEST_full_qtt.drop(['insee','date'],axis=1)

In [ ]:
X_TEST = scaler.transform(df_TEST_full_qtt) # Normalise les données
Y_PRED = nnetOpt.predict(X_TEST) # Ici , changer nnetOpt par votre modèle de prédiction entraîné

In [ ]:
Annex.generate_submission_file('submission_17nov2017_10h30.csv', Y_PRED) #changer le nom si vous voulez